# Crawler -- sources

Date: 2024/05/02-2024/05/06

## 前処理
- Beautiful SoapでPDFファイルのリンクリスト作成
- PyMuPDFで各PDFファイルのText抽出しSQLiteへ格納
- 同時にspaCyでNERを行い、その結果をSQLiteへ格納

```
Table "sources" <= 経産省、総務省、XX株式会社といった白書やIR資料を提供している団体名
source_id(primary key), base_url, homepage_url, org, doc, doc_url, category

Table "links" <= Beautiful Soapで抽出したPDFファイルのURl
link_id, path, title, source_id fk(sources:source_id)

Table "texts" <= PyMuPDFで抽出したテキスト
link_idf fk(links:link_id), page, text

Table "named_entities" <= spaCyで抽出したNamed Entity
link_id fk(links:link_id), page, named_entity, label
```

## APIサーバ (Flaskで実装)

#### /sources

\[{source_id: \<source_id\>, base_url: \<base_url\>, homepage_url: \<homepage_url\>, org: \<base_url\>, doc: \<doc\>, doc_url: \<doc_url\>, category: \<category\>},...\]

PDFファイル検索対象となる団体のリストを返信（今回は経済産業省のみ）。

#### /search?base_url=\<base_url\>&keywords=\<keywords\>

\[{link_id: \<link\>, title: \<title\>, page: \<page\>, text: \<text\>, spans: {keyword: \[\[\<start\>, \<end\>\],...\],...}\]

該当キーワードを含むPDFファイルのパスとページ番号を返信。base_urlを指定しない場合、全ての団体を検索対象とする。

#### /highlight?link_id=\<link_id\>&page=\<page\>&keywords=\<keywords\>&all_pages=\<all_pages\>

該当キーワードをハイライトしてPDFを返送。指定ページの前後で合計３ページ分を返信。

In [1]:
# 2023年度(令和5年)白書

ORGS_JAPANESE_GOV = [
    ["経済産業省",
     "通商白書 2023",
     "https://www.meti.go.jp",
     "https://www.meti.go.jp",
     "https://www.meti.go.jp/report/tsuhaku2023/whitepaper_2023.html"],
    ["総務省",
     "情報通信白書 2023",
     "https://www.soumu.go.jp",
     "https://www.soumu.go.jp/johotsusintokei/whitepaper/ja/r05/pdf",
     "https://www.soumu.go.jp/johotsusintokei/whitepaper/ja/r05/pdf/index.html"],
    ["防衛省",
     "防衛白書 2023",
     "https://www.mod.go.jp/",
     "http://www.clearing.mod.go.jp/hakusho_data/2023/pdf",
     "http://www.clearing.mod.go.jp/hakusho_data/2023/pdf/index.html"]
]

ORGS_LOCAL_GOVERNMENT = [
    ["横浜市",
     "横浜市中期計画 2022～2025",
     "https://www.city.yokohama.lg.jp",
     "https://www.city.yokohama.lg.jp/city-info/seisaku/hoshin/4kanen/2022-2025/chuki2022-.files",
     "https://www.city.yokohama.lg.jp/city-info/seisaku/hoshin/4kanen/2022-2025/chuki2022-.html"
    ],
    ["川崎市",
     "川崎市総合計画 2022〜2025",
     "https://www.city.kawasaki.jp/index.html",
     "https://www.city.kawasaki.jp/170/page",
     "https://www.city.kawasaki.jp/170/page/0000138364.html"
    ]
]

ORGS = {"日本政府": ORGS_JAPANESE_GOV, "自治体": ORGS_LOCAL_GOVERNMENT}

In [2]:
%run ./lib.py

In [3]:
import sqlite3

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS sources')
    cur.execute('CREATE TABLE sources (source_id INTEGER PRIMARY KEY AUTOINCREMENT, \
    base_url TEXT, homepage_url TEXT, org TEXT, doc TEXT, doc_url TEXT, category TEXT, \
    UNIQUE(homepage_url, org, doc, doc_url, category))')
    for category, orgs in ORGS.items():
        for o in orgs:
            org = o[0]
            doc = o[1]
            homepage_url =o[2]
            base_url = o[3]
            doc_url = o[4]
            cur.execute(f'INSERT INTO sources (base_url, homepage_url, org, doc, doc_url, category) \
            VALUES ("{base_url}", "{homepage_url}", "{org}", "{doc}", "{doc_url}", "{category}")')